In [1]:
import pandas as pd
import numpy as np

# Import the first CSV file
cleaned_df = pd.read_csv('ufc_cleaned.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
cleaned_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_dob,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_dob,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.
0,Cory Sandhagen,Deiveson Figueiredo,0,0,0.62,0.66,1.00,0.40,0,0,1,1,307,31,0,0,0.79,1.00,1.00,0.25,0,0,0,0,217,4,0.62,0.66,0.79,1.00,KO/TKO,Dan Miragliotta,1,2025-05-03,54,87,6,9,92,149,8,11,2,2,2,5,31,39,2,2,53,78,3,3,1,1,1,4,33,65,4,6,2,2,0,0,19,20,2,3,24,43,4,7,0,0,2,2,30,44,0,0,22,30,1,1,1,1,0,0,8,8,1,1,10,13,1,1,0,0,1,1,21,26,0,0,71.0,135.0,70.0,Switch,1992-04-20,5.02,0.44,3.46,0.56,1.19,0.32,0.63,0.3,65.0,135.0,68.0,Orthodox,1987-12-18,2.91,0.54,3.55,0.49,1.62,0.34,0.60,1.4
1,Reinier de Ridder,Bo Nickal,1,0,0.68,0.38,0.00,1.00,0,0,0,0,129,144,0,0,0.50,0.33,0.00,1.00,0,0,0,0,112,120,0.68,0.38,0.50,0.33,KO/TKO,Mike Beltran,1,2025-05-03,22,32,8,21,91,104,24,40,0,3,1,1,5,10,2,6,43,49,17,24,0,2,1,1,9,18,2,13,13,14,4,5,0,0,2,3,7,15,3,14,15,17,5,7,0,0,0,0,2,7,1,5,3,3,1,1,0,0,0,0,1,5,0,4,4,5,2,2,0,0,0,0,76.0,185.0,78.0,Southpaw,1990-09-07,3.28,0.57,2.69,0.38,6.89,0.46,1.00,2.0,73.0,185.0,76.0,Southpaw,1996-01-14,2.65,0.63,1.86,0.64,3.61,0.50,0.00,3.6
2,Santiago Ponzinibbio,Daniel Rodriguez,0,1,0.44,0.39,0.12,NaN,0,0,0,0,44,7,0,0,0.42,0.43,0.00,NaN,0,0,0,0,2,7,0.44,0.39,0.42,0.43,KO/TKO,Kevin MacDonald,0,2025-05-03,49,111,66,166,50,115,70,170,1,8,0,0,15,35,35,81,15,35,35,81,0,2,0,0,31,83,39,131,6,16,20,25,12,12,7,10,49,111,64,164,0,0,1,1,0,0,1,1,5,19,20,61,3,9,13,16,7,7,2,4,15,35,34,80,0,0,1,1,0,0,0,0,72.0,170.0,73.0,Orthodox,1986-09-26,4.80,0.41,4.61,0.61,0.56,0.32,0.71,0.0,73.0,170.0,74.0,Southpaw,1986-12-31,7.50,0.49,5.47,0.58,0.59,0.54,0.57,0.1
3,Montel Jackson,Daniel Marcos,0,0,0.45,0.53,0.25,0.42,1,0,0,0,217,345,0,0,0.33,0.57,0.25,1.00,0,0,0,0,127,86,0.45,0.53,0.33,0.57,Decision - Unanimous,Gary Copeland,1,2025-05-03,30,66,17,32,67,106,24,43,2,8,3,7,4,12,4,7,22,31,7,11,1,4,1,1,25,58,5,18,1,3,5,7,4,5,7,7,27,62,13,27,3,4,3,4,0,0,1,1,2,8,2,4,1,2,1,2,1,2,1,1,3,11,3,6,1,1,1,1,0,0,0,0,70.0,135.0,75.0,Southpaw,1992-04-24,3.39,0.53,1.42,0.63,3.41,0.71,0.69,0.3,67.0,135.0,69.0,Orthodo

In [ ]:
#calculate fighters age relative to event
cleaned_df['event_date'] = pd.to_datetime(cleaned_df['event_date'])
cleaned_df['p1_dob'] = pd.to_datetime(cleaned_df['p1_dob'])
cleaned_df['p2_dob'] = pd.to_datetime(cleaned_df['p2_dob'])

cleaned_df['p1_age_at_event'] = (cleaned_df['event_date'] - cleaned_df['p1_dob']).dt.days / 365.25
cleaned_df['p2_age_at_event'] = (cleaned_df['event_date'] - cleaned_df['p2_dob']).dt.days / 365.25

cleaned_df = cleaned_df.drop(columns=['p1_dob', 'p2_dob'])

#compute physical differences
cleaned_df['height_diff'] = cleaned_df['p1_height'] - cleaned_df['p2_height']
cleaned_df['reach_diff']  = cleaned_df['p1_reach']  - cleaned_df['p2_reach']
cleaned_df['weight_diff'] = cleaned_df['p1_weight'] - cleaned_df['p2_weight']
cleaned_df['age_diff']    = cleaned_df['p1_age_at_event'] - cleaned_df['p2_age_at_event']

#compute fighting metric differences
metrics = ['SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']

for m in metrics:
  p1_col = f'p1_{m}'
  p2_col = f'p2_{m}'
  diff_col = f'{m.replace(" ", "").replace(".", "").lower()}_diff'
  cleaned_df[diff_col] = cleaned_df[p1_col] - cleaned_df[p2_col]


#find says days since last fight
from collections import defaultdict

def calculate_days_since_last_fight(df):
    df = df.sort_values('event_date').reset_index(drop=True)
    fighter_dates = defaultdict(list)
    
    # Pre-process all fight dates per fighter
    for _, row in df.iterrows():
        fighter_dates[row['p1_fighter']].append(row['event_date'])
        fighter_dates[row['p2_fighter']].append(row['event_date'])
    
    # Find previous fight dates
    def get_prev_date(fighter, current_date):
        dates = [d for d in fighter_dates[fighter] if d < current_date]
        return max(dates) if dates else None
    
    df['p1_prev_date'] = df.apply(lambda x: get_prev_date(x['p1_fighter'], x['event_date']), axis=1)
    df['p2_prev_date'] = df.apply(lambda x: get_prev_date(x['p2_fighter'], x['event_date']), axis=1)
    
    # Calculate day differences
    df['p1_days_since_last_fight'] = (df['event_date'] - df['p1_prev_date']).dt.days
    df['p2_days_since_last_fight'] = (df['event_date'] - df['p2_prev_date']).dt.days
    
    # Calculate the difference (similar to your other diff metrics)
    df['days_since_last_fight_diff'] = df['p1_days_since_last_fight'] - df['p2_days_since_last_fight']
    
    # Drop temporary columns if not needed
    df = df.drop(columns=['p1_prev_date', 'p2_prev_date'])
    
    return df

cleaned_df = calculate_days_since_last_fight(cleaned_df)
  
  
# Sort the dataframe by event_date to ensure chronological order
cleaned_df = cleaned_df.sort_values(by='event_date')

# Initialize fighter record tracker
fighter_records = {}

# Create new columns
cleaned_df['p1_wins'] = 0
cleaned_df['p1_losses'] = 0
cleaned_df['p1_total'] = 0
cleaned_df['p2_wins'] = 0
cleaned_df['p2_losses'] = 0
cleaned_df['p2_total'] = 0

for index, row in cleaned_df.iterrows(): # calculate each fighters recoord relative to when the match happened
    p1 = row['p1_fighter']
    p2 = row['p2_fighter']
    winner = row['winner']
    
    # Get current records (wins, losses, total fights)
    p1_record = fighter_records.get(p1, (0, 0, 0))
    p2_record = fighter_records.get(p2, (0, 0, 0))
    
    # Assign pre-fight records
    cleaned_df.at[index, 'p1_wins'] = p1_record[0]
    cleaned_df.at[index, 'p1_losses'] = p1_record[1]
    cleaned_df.at[index, 'p1_total'] = p1_record[0] + p1_record[1]
    
    cleaned_df.at[index, 'p2_wins'] = p2_record[0]
    cleaned_df.at[index, 'p2_losses'] = p2_record[1]
    cleaned_df.at[index, 'p2_total'] = p2_record[0] + p2_record[1]
    
    # Update records after fight based on the winner
    if winner == 1:  # p1 won
        fighter_records[p1] = (p1_record[0] + 1, p1_record[1], p1_record[0] + p1_record[1] + 1)
        fighter_records[p2] = (p2_record[0], p2_record[1] + 1, p2_record[0] + p2_record[1] + 1)
    else:  # p2 won
        fighter_records[p1] = (p1_record[0], p1_record[1] + 1, p1_record[0] + p1_record[1] + 1)
        fighter_records[p2] = (p2_record[0] + 1, p2_record[1], p2_record[0] + p2_record[1] + 1)

# Calculate differences
cleaned_df['win_diff'] = cleaned_df['p1_wins'] - cleaned_df['p2_wins']
cleaned_df['loss_diff'] = cleaned_df['p1_losses'] - cleaned_df['p2_losses']
cleaned_df['total_diff'] = cleaned_df['p1_total'] - cleaned_df['p2_total']


# Initialize columns
cleaned_df['p1_win_streak'] = 0
cleaned_df['p2_win_streak'] = 0

# Initialize win streak tracker
win_streaks = {}

# Calculate win streaks relative to when match happened
for idx, row in cleaned_df.iterrows():
    p1, p2 = row['p1_fighter'], row['p2_fighter']
    winner = row['winner']
    
    # Assign pre-fight streaks
    cleaned_df.at[idx, 'p1_win_streak'] = win_streaks.get(p1, 0)
    cleaned_df.at[idx, 'p2_win_streak'] = win_streaks.get(p2, 0)
    
    # Update streaks post-fight based on actual winner
    if winner == 1:  # p1 won
        win_streaks[p1] = win_streaks.get(p1, 0) + 1
        win_streaks[p2] = 0  # Reset losing fighter's streak
    else:  # p2 won
        win_streaks[p1] = 0  # Reset losing fighter's streak
        win_streaks[p2] = win_streaks.get(p2, 0) + 1

# Define columns to process for both fighters
p1_cols = ['p1_SLpM','p1_Str. Acc.','p1_SApM','p1_Str. Def','p1_TD Avg.','p1_TD Acc.','p1_TD Def.','p1_Sub. Avg.']
p2_cols = ['p2_SLpM','p2_Str. Acc.','p2_SApM','p2_Str. Def','p2_TD Avg.','p2_TD Acc.','p2_TD Def.','p2_Sub. Avg.']

# Create age-adjusted versions for all specified columns
for p1_col, p2_col in zip(p1_cols, p2_cols):
    cleaned_df[p1_col.replace('p1_', 'p1_age_adjusted_')] = cleaned_df[p1_col] * (1/cleaned_df['p1_age_at_event'])
    cleaned_df[p2_col.replace('p2_', 'p2_age_adjusted_')] = cleaned_df[p2_col] * (1/cleaned_df['p2_age_at_event'])

cleaned_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.
0,Scott Morris,Sean Daugherty,0,0,1.00,0.00,1.0,NaN,1,0,0,0,0,0,0,0,1.00,0.00,1.0,NaN,1,0,0,0,0,0,1.00,0.00,1.00,0.00,Submission,John McCarthy,1,1994-03-11,1,1,0,4,2,2,1,5,1,1,0,0,1,1,0,4,2,2,1,5,1,1,0,0,1,1,0,2,0,0,0,0,0,0,0,2,0,0,0,3,1,1,0,1,0,0,0,0,1,1,0,2,0,0,0,0,0,0,0,2,0,0,0,3,1,1,0,1,0,0,0,0,70.0,210.0,NaN,Orthodox,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,72.0,175.0,NaN,NaN,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,NaN,18.266940,-2.0,NaN,35.0,NaN,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,NaN,0.000000
14,Patrick Smith,Scott Morris,0,0,0.76,NaN,NaN,0.0,0,0,0,0,0,0,0,0,0.76,NaN,NaN,0.0,0,0,0,0,0,0,0.76,NaN,0.76,NaN,KO/TKO,John McCarthy,1,1994-03-11,13,17,0,0,19,25,0,0,0,0,0,1,13,17,0,0,19,25,0,0,0,0,0,1,12,16,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,12,16,0,0,12,16,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,12,16,0,0,74.0,225.0,NaN,Orthodox,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,70.0,210.0,NaN,Orthodox,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,30.535250,NaN,4.0,NaN,15.0,NaN,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,NaN,NaN,NaN,0,0,0,1,0,1,-1,0,-1,0,1,0.000000,NaN,0.000000,NaN,0.000000,

In [3]:
#calculate all the ema

# Ensure data is sorted by event_date
df = cleaned_df.sort_values(by='event_date')

# List of features to calculate EMA for
features = [
    'KD', 'SIG_STR_PCT', 'TD_PCT', 'SUB_ATT', 'REV', 'CTRL',
    'R1_KD', 'R1_SIG_STR_PCT', 'R1_TD_PCT', 'R1_SUB_ATT', 'R1_REV', 'R1_CTRL',
    'SIG_STR_PCT_DETAILED', 'R1_SIG_STR_PCT_DETAILED',
    'SIG_STR_LANDED', 'SIG_STR_ATTEMPTED', 'TOTAL_STR_LANDED', 'TOTAL_STR_ATTEMPTED',
    'TD_LANDED', 'TD_ATTEMPTED',
    'R1_SIG_STR_LANDED', 'R1_SIG_STR_ATTEMPTED', 'R1_TOTAL_STR_LANDED', 'R1_TOTAL_STR_ATTEMPTED',
    'R1_TD_LANDED', 'R1_TD_ATTEMPTED',
    'HEAD_LANDED', 'HEAD_ATTEMPTED', 'BODY_LANDED', 'BODY_ATTEMPTED',
    'LEG_LANDED', 'LEG_ATTEMPTED',
    'DISTANCE_LANDED', 'DISTANCE_ATTEMPTED', 'CLINCH_LANDED', 'CLINCH_ATTEMPTED',
    'GROUND_LANDED', 'GROUND_ATTEMPTED',
    'R1_HEAD_LANDED', 'R1_HEAD_ATTEMPTED', 'R1_BODY_LANDED', 'R1_BODY_ATTEMPTED',
    'R1_LEG_LANDED', 'R1_LEG_ATTEMPTED',
    'R1_DISTANCE_LANDED', 'R1_DISTANCE_ATTEMPTED', 'R1_CLINCH_LANDED', 'R1_CLINCH_ATTEMPTED',
    'R1_GROUND_LANDED', 'R1_GROUND_ATTEMPTED'
]

# Process p1 fighter data
p1_cols = {f'p1_{feat}': feat for feat in features}
p1_stats = df[['p1_fighter', 'p2_fighter', 'event_date'] + list(p1_cols.keys())].copy()
p1_stats['fighter'] = p1_stats['p1_fighter']
p1_stats['position'] = 'p1'
p1_stats = p1_stats.rename(columns=p1_cols)

# Process p2 fighter data
p2_cols = {f'p2_{feat}': feat for feat in features}
p2_stats = df[['p1_fighter', 'p2_fighter', 'event_date'] + list(p2_cols.keys())].copy()
p2_stats['fighter'] = p2_stats['p2_fighter']
p2_stats['position'] = 'p2'
p2_stats = p2_stats.rename(columns=p2_cols)

# Combine fighter stats
columns_to_keep = ['fighter', 'position', 'event_date', 'p1_fighter', 'p2_fighter'] + list(features)
fighter_stats = pd.concat([p1_stats[columns_to_keep], p2_stats[columns_to_keep]], ignore_index=True)

# Sort by fighter and date
fighter_stats = fighter_stats.sort_values(by=['fighter', 'event_date']).copy()

# Convert all feature columns to numeric
for feat in features:
    fighter_stats[feat] = pd.to_numeric(fighter_stats[feat], errors='coerce')

# Calculate EMAs for all features grouped by fighter
ema_dfs = []
for feat in features:
    ema_series = fighter_stats.groupby('fighter')[feat].transform(
        lambda x: x.ewm(span=3).mean()
    )
    ema_dfs.append(pd.DataFrame({f'{feat}_EMA': ema_series}))

# Join all EMAs at once
ema_df = pd.concat(ema_dfs, axis=1)
fighter_stats = pd.concat([fighter_stats, ema_df], axis=1)

# Create final result DataFrame
result_df = df.copy()

# Initialize EMA columns in the result DataFrame at once
ema_columns = {}
for feat in features:
    ema_columns[f'p1_{feat}_EMA'] = np.nan
    ema_columns[f'p2_{feat}_EMA'] = np.nan

result_ema_df = pd.DataFrame(ema_columns, index=result_df.index)
result_df = pd.concat([result_df, result_ema_df], axis=1)

# Add EMA values to result DataFrame
for idx, row in result_df.iterrows():
    date = row['event_date']
    
    # Get p1 fighter EMA values from previous fights
    p1_fighter = row['p1_fighter']
    p1_prev = fighter_stats[(fighter_stats['fighter'] == p1_fighter) & 
                           (fighter_stats['event_date'] < date)]
    
    if not p1_prev.empty:
        p1_latest = p1_prev.sort_values('event_date').iloc[-1]
        for feat in features:
            ema_col = f'{feat}_EMA'
            if ema_col in p1_latest and not pd.isna(p1_latest[ema_col]):
                result_df.loc[idx, f'p1_{feat}_EMA'] = p1_latest[ema_col]
    
    # Get p2 fighter EMA values from previous fights
    p2_fighter = row['p2_fighter']
    p2_prev = fighter_stats[(fighter_stats['fighter'] == p2_fighter) & 
                           (fighter_stats['event_date'] < date)]
    
    if not p2_prev.empty:
        p2_latest = p2_prev.sort_values('event_date').iloc[-1]
        for feat in features:
            ema_col = f'{feat}_EMA'
            if ema_col in p2_latest and not pd.isna(p2_latest[ema_col]):
                result_df.loc[idx, f'p2_{feat}_EMA'] = p2_latest[ema_col]

# Defragment the final DataFrame
result_df = result_df.copy()


In [ ]:
# Drop all in match data

result_df.sort_values(by='event_date', ascending=False, inplace=True)
columns_to_drop = [
    'p1_KD', 'p2_KD', 'p1_SIG_STR_PCT', 'p2_SIG_STR_PCT', 'p1_TD_PCT', 'p2_TD_PCT', 
    'p1_SUB_ATT', 'p2_SUB_ATT', 'p1_REV', 'p2_REV', 'p1_CTRL', 'p2_CTRL', 
    'p1_R1_KD', 'p2_R1_KD', 'p1_R1_SIG_STR_PCT', 'p2_R1_SIG_STR_PCT', 
    'p1_R1_TD_PCT', 'p2_R1_TD_PCT', 'p1_R1_SUB_ATT', 'p2_R1_SUB_ATT', 
    'p1_R1_REV', 'p2_R1_REV', 'p1_R1_CTRL', 'p2_R1_CTRL', 
    'p1_SIG_STR_PCT_DETAILED', 'p2_SIG_STR_PCT_DETAILED', 
    'p1_R1_SIG_STR_PCT_DETAILED', 'p2_R1_SIG_STR_PCT_DETAILED',
    'p1_SIG_STR_LANDED', 'p1_SIG_STR_ATTEMPTED', 'p2_SIG_STR_LANDED', 'p2_SIG_STR_ATTEMPTED',
    'p1_TOTAL_STR_LANDED', 'p1_TOTAL_STR_ATTEMPTED', 'p2_TOTAL_STR_LANDED', 'p2_TOTAL_STR_ATTEMPTED',
    'p1_TD_LANDED', 'p1_TD_ATTEMPTED', 'p2_TD_LANDED', 'p2_TD_ATTEMPTED',
    'p1_R1_SIG_STR_LANDED', 'p1_R1_SIG_STR_ATTEMPTED', 'p2_R1_SIG_STR_LANDED', 'p2_R1_SIG_STR_ATTEMPTED',
    'p1_R1_TOTAL_STR_LANDED', 'p1_R1_TOTAL_STR_ATTEMPTED', 'p2_R1_TOTAL_STR_LANDED', 'p2_R1_TOTAL_STR_ATTEMPTED',
    'p1_R1_TD_LANDED', 'p1_R1_TD_ATTEMPTED', 'p2_R1_TD_LANDED', 'p2_R1_TD_ATTEMPTED',
    'p1_HEAD_LANDED', 'p1_HEAD_ATTEMPTED', 'p2_HEAD_LANDED', 'p2_HEAD_ATTEMPTED',
    'p1_BODY_LANDED', 'p1_BODY_ATTEMPTED', 'p2_BODY_LANDED', 'p2_BODY_ATTEMPTED',
    'p1_LEG_LANDED', 'p1_LEG_ATTEMPTED', 'p2_LEG_LANDED', 'p2_LEG_ATTEMPTED',
    'p1_DISTANCE_LANDED', 'p1_DISTANCE_ATTEMPTED', 'p2_DISTANCE_LANDED', 'p2_DISTANCE_ATTEMPTED',
    'p1_CLINCH_LANDED', 'p1_CLINCH_ATTEMPTED', 'p2_CLINCH_LANDED', 'p2_CLINCH_ATTEMPTED',
    'p1_GROUND_LANDED', 'p1_GROUND_ATTEMPTED', 'p2_GROUND_LANDED', 'p2_GROUND_ATTEMPTED',
    'p1_R1_HEAD_LANDED', 'p1_R1_HEAD_ATTEMPTED', 'p2_R1_HEAD_LANDED', 'p2_R1_HEAD_ATTEMPTED',
    'p1_R1_BODY_LANDED', 'p1_R1_BODY_ATTEMPTED', 'p2_R1_BODY_LANDED', 'p2_R1_BODY_ATTEMPTED',
    'p1_R1_LEG_LANDED', 'p1_R1_LEG_ATTEMPTED', 'p2_R1_LEG_LANDED', 'p2_R1_LEG_ATTEMPTED',
    'p1_R1_DISTANCE_LANDED', 'p1_R1_DISTANCE_ATTEMPTED', 'p2_R1_DISTANCE_LANDED', 'p2_R1_DISTANCE_ATTEMPTED',
    'p1_R1_CLINCH_LANDED', 'p1_R1_CLINCH_ATTEMPTED', 'p2_R1_CLINCH_LANDED', 'p2_R1_CLINCH_ATTEMPTED',
    'p1_R1_GROUND_LANDED', 'p1_R1_GROUND_ATTEMPTED', 'p2_R1_GROUND_LANDED', 'p2_R1_GROUND_ATTEMPTED',
    'event_date', 'method'
]

# Drop the columns
result_df = result_df.drop(columns=columns_to_drop)

result_df.head()

,p1_fighter,p2_fighter,referee,winner,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.,p1_KD_EMA,p2_KD_EMA,p1_SIG_STR_PCT_EMA,p2_SIG_STR_PCT_EMA,p1_TD_PCT_EMA,p2_TD_PCT_EMA,p1_SUB_ATT_EMA,p2_SUB_ATT_EMA,p1_REV_EMA,p2_REV_EMA,p1_CTRL_EMA,p2_CTRL_EMA,p1_R1_KD_EMA,p2_R1_KD_EMA,p1_R1_SIG_STR_PCT_EMA,p2_R1_SIG_STR_PCT_EMA,p1_R1_TD_PCT_EMA,p2_R1_TD_PCT_EMA,p1_R1_SUB_ATT_EMA,p2_R1_SUB_ATT_EMA,p1_R1_REV_EMA,p2_R1_REV_EMA,p1_R1_CTRL_EMA,p2_R1_CTRL_EMA,p1_SIG_STR_PCT_DETAILED_EMA,p2_SIG_STR_PCT_DETAILED_EMA,p1_R1_SIG_STR_PCT_DETAILED_EMA,p2_R1_SIG_STR_PCT_DETAILED_EMA,p1_SIG_STR_LANDED_EMA,p2_SIG_STR_LANDED_EMA,p1_SIG_STR_ATTEMPTED_EMA,p2_SIG_STR_ATTEMPTED_EMA,p1_TOTAL_STR_LANDED_EMA,p2_TOTAL_STR_LANDED_EMA,p1_TOTAL_STR_ATTEMPTED_EMA,p2_TOTAL_STR_ATTEMPTED_EMA,p1_TD_LANDED_EMA,p2_TD_LANDED_EMA,p1_TD_ATTEMPTED_EMA,p2_TD_ATTEMPTED_EMA,p1_R1_SIG_STR_LANDED_EMA,p2_R1_SIG_STR_LANDED_EMA,p1_R1_SIG_STR_ATTEMPTED_EMA,p2_R1_SIG_STR_ATTEMPTED_EMA,p1_R1_TOTAL_STR_LANDED_EMA,p2_R1_TOTAL_STR_LANDED_EMA,p1_R1_TOTAL_STR_ATTEMPTED_EMA,p2_R1_TOTAL_STR_ATTEMPTED_EMA,p1_R1_TD_LANDED_EMA,p2_R1_TD_LANDED_EMA,p1_R1_TD_ATTEMPTED_EMA,p2_R1_TD_ATTEMPTED_EMA,p1_HEAD_LANDED_EMA,p2_HEAD_LANDED_EMA,p1_HEAD_ATTEMPTED_EMA,p2_HEAD_ATTEMPTED_EMA,p1_BODY_LANDED_EMA,p2_BODY_LANDED_EMA,p1_BODY_ATTEMPTED_EMA,p2_BODY_ATTEMPTED_EMA,p1_LEG_LANDED_EMA,p2_LEG_LANDED_EMA,p1_LEG_ATTEMPTED_EMA,p2_LEG_ATTEMPTED_EMA,p1_DISTANCE_LANDED_EMA,p2_DISTANCE_LANDED_EMA,p1_DISTANCE_ATTEMPTED_EMA,p2_DISTANCE_ATTEMPTED_EMA,p1_CLINCH_LANDED_EMA,p2_CLINCH_LANDED_EMA,p1_CLINCH_ATTEMPTED_EMA,p2_CLINCH_ATTEMPTED_EMA,p1_GROUND_LANDED_EMA,p2_GROUND_LANDED_EMA,p1_GROUND_ATTEMPTED_EMA,p2_GROUND_ATTEMPTED_EMA,p1_R1_HEAD_LANDED_EMA,p2_R1_HEAD_LANDED_EMA,p1_R1_HEAD_ATTEMPTED_EMA,p2_R1_HEAD_ATTEMPTED_EMA,p1_R1_BODY_LANDED_EMA,p2_R1_BODY_LANDED_EMA,p1_R1_BODY_ATTEMPTED_EMA,p2_R1_BODY_ATTEMPTED_EMA,p1_R1_LEG_LANDED_EMA,p2_R1_LEG_LANDED_EMA,p1_R1_LEG_ATTEMPTED_EMA,p2_R1_LEG_ATTEMPTED_EMA,p1_R1_DISTANCE_LANDED_EMA,p2_R1_DISTANCE_LANDED_EMA,p1_R1_DISTANCE_ATTEMPTED_EMA,p2_R1_DISTANCE_ATTEMPTED_EMA,p1_R1_CLINCH_LANDED_EMA,p2_R1_CLINCH_LANDED_EMA,p1_R1_CLINCH_ATTEMPTED_EMA,p2_R1_CLINCH_ATTEMPTED_EMA,p1_R1_GROUND_LANDED_EMA,p2_R1_GROUND_LANDED_EMA,p1_R1_GROUND_ATTEMPTED_EMA,p2_R1_GROUND_ATTEMPTED_EMA
8175,Cory Sandhagen,Deiveson Figueiredo,Dan Miragliotta,1,71.0,135.0,70.0,Switch,5.02,0.44,3.46,0.56,1.19,0.32,0.63,0.3,65.0,135.0,68.0,Orthodox,2.91,0.54,3.55,0.49,1.62,0.34,0.60,1.4,33.034908,37.374401,6.0,2.0,0.0,-4.339493,2.11,-0.10,-0.09,0.07,-0.43,-0.02,0.03,-1.1,273.0,161.0,112.0,10,4,14,13,5,18,-3,-1,-4,0,0,0.151960,0.077861,0.013319,0.014448,0.104738,0.094985,0.016952,0.013111,0.036023,0.043345,0.009687,0.009097,0.019071,0.016054,0.009081,0.037459,0.012025,0.299520,0.427808,0.499030,0.295141,0.250166,0.266130,0.319192,0.032290,0.000000,369.136117,174.851985,0.008057,0.001953,0.434544,0.846666,0.664548,0.467068,0.016114,0.037895,0.000000,0.000000,62.763841,98.073334,0.427808,0.499030,0.434544,0.846666,73.462980,44.916534,168.352377,94.296907,112.881890,50.330579,222.104682,99.836223,2.2201

In [5]:

result_df.to_csv('ufc_features.csv', index=False)